In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
import keras_preprocessing
from keras_preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [ ]:
training_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dir = "./train/train"
train_gen = training_datagen.flow_from_directory(train_dir, target_size=(150, 150), class_mode="categorical")       

In [ ]:
val_dir = "./train/val"
val_gen = validation_datagen.flow_from_directory(val_dir, target_size=(150, 150), class_mode="categorical")

In [ ]:
# split directory
train_split_dir = r"./train/train/split"

# printing the number of split tomatoes in train dataset
number_split_train = len(os.listdir(train_split_dir))
print("total training split images:", number_split_train)

# not split directory
train_not_split_dir = r"./train/train/no_split"

# printing the number of split tomatoes in train dataset
number_not_split_train = len(os.listdir(train_not_split_dir))
print("total training not split images:", number_not_split_train)

In [ ]:
split_names = os.listdir(train_split_dir)
split_names[:10]
not_split_names = os.listdir(train_not_split_dir)
not_split_names[:10]

In [ ]:
model = tf.keras.models.Sequential([
    # first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # third convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # fourth convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten before feeding into Dense neural network. 
    tf.keras.layers.Flatten(),
    # 512 neurons in the hidden layer
    tf.keras.layers.Dense(512, activation="relu"),
    # 2 categories of tomatoes 
    # softmas takes a set of values and effectively picks the biggest one. for example if the output layer has
    # [0.1,0.1,0.5,0.2,0.1], it will take it and turn it into [0,0,1,0,0]
    tf.keras.layers.Dense(2, activation="softmax")
]);

In [ ]:
model.summary()

In [ ]:
# implementing a callback function to terminate training once training reaches 98% accuracy for validation data

validation_accuracy = 0.98

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc') is not None and logs.get('val_acc') >= validation_accuracy):
            print("\nReached desired validation accuracy, so cancelling training")
            self.model.stop_training=True
            
callbacks = myCallback()

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
tomato_model = model.fit(train_gen, epochs=100, validation_data=val_gen, verbose=1, callbacks = [callbacks], workers=10)

filepath = "./saved_model"
tf.keras.models.save_model(
    model,
    filepath,
    overwrite=True,
    include_optimizer=True,
    save_format="tf",
    signatures=None
)

model.save("tomato.h5")

In [ ]:
import numpy as np
split_image = "./testing_images/images - 2023-02-03T172532.051.jpg"

img = image.load_img(split_image, target_size = (150, 150))
array = image.img_to_array(img)
x = np.expand_dims(array, axis=0)

vimage = np.vstack([x])
model.predict(vimage)

In [ ]:
import numpy as np
not_split_image = "./testing_images/images (62).jpg"

img = image.load_img(not_split_image, target_size = (150, 150))
array = image.img_to_array(img)
x = np.expand_dims(array, axis=0)

vimage = np.vstack([x])
model.predict(vimage)

In [5]:
import numpy as np
image_dir = "./testing_images"

def fruit_prediction(image_dir):
    img_list = os.listdir(image_dir)
    for tomato in img_list:
        path = os.path.join(image_dir, tomato)
        img = image.load_img(path, target_size = (150, 150))
        array = image.img_to_array(img)
        x = np.expand_dims(array, axis=0)

        vimage = np.vstack([x])
        img_classification = model.predict(vimage)
        print(img_classification, tomato)

fruit_prediction(image_dir)